In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pickle


In [ ]:
avilable_languages = ['eng','fin','jap']
current_language = 'fin'

In [ ]:
# import training data
dft = pd.read_csv('../data/dft_'+current_language+'.csv')

# import validation data
dfv = pd.read_csv('../data/dfv_'+current_language+'.csv')

In [ ]:
dft

## Bag of Words

### Functions to create BoW vectors

In [ ]:
def make_bow_vector(sentence, countt):
    vec = np.zeros(len(countt))
    for word in sentence:
        vec[countt[word]] += 1
    return vec

def ans_freq(que, doc):

    freq = [x for x in que if x in doc]
    freq = len(freq)/len(que)
    return freq

def freq_words_in_text (df_t,df_v):
    for df in [df_t,df_v]:
        frequency = []
        for question, answer in zip(df['question_text_tokenized'], df['document_plaintext_tokenized']):
            frequency.append(ans_freq(eval(question), eval(answer)))

        df['word_frequency_score'] = frequency

### Combine and flatten

In [ ]:
t_comb2 = [str(que)+ " "+ str(doc) for que, doc in zip(dft["question_text"],dft["document_plaintext"])]
v_comb2 = [str(que)+ " "+ str(doc) for que, doc in zip(dfv["question_text"],dfv["document_plaintext"])]

In [ ]:
t_comb2.extend(v_comb2)

In [ ]:
CountVec = CountVectorizer(ngram_range=(1,1))
Count_data = CountVec.fit_transform(t_comb2)

In [ ]:
freq_words_in_text(dft,dfv)

In [ ]:
ct = Count_data.toarray()[:len(dft)]
cv = Count_data.toarray()[len(dft):]

In [ ]:
ct = [l.tolist() for l in Count_data.toarray()[:len(dft)]]

In [ ]:
cv = [l.tolist() for l in Count_data.toarray()[len(dft):]]

In [ ]:
t_wf = dft['word_frequency_score'].values
v_wf = dfv['word_frequency_score'].values

In [ ]:
xt = []
for index in range(len(dft)):
    xt.append(np.append(t_wf[index],ct[index]))
    
xv = []
for index in range(len(dfv)):
    xv.append(np.append(v_wf[index],cv[index]))

In [ ]:
X_train = np.array(xt)
X_val = np.array(xv)
y_train = dft.label.values
y_val = dfv.label.values

In [ ]:
clf = LogisticRegression(C=1000, penalty='l1', random_state=1, solver='liblinear').fit(X_train, y_train)

In [ ]:
val_pred = clf.predict(X_val)
accuracy_score(val_pred, y_val)

In [ ]:
train_pred = clf.predict(X_train)
accuracy_score(train_pred, y_train)

In [ ]:
#filename = '../../models/logres_model_'+current_language+'.sav'
#pickle.dump(clf, open(filename, 'wb'))

In [ ]:
len(val_pred)

In [ ]:
dft['logres_pred']=train_pred
dfv['logres_pred']=val_pred


In [ ]:
dft

In [ ]:
dft.to_csv('../data/dft_'+current_language+'.csv',index=False) 

In [ ]:
dfv.to_csv('../data/dfv_'+current_language+'.csv',index=False) 